### Матричные факторизации

В данной работе вам предстоит познакомиться с практической стороной матричных разложений.
Работа поделена на 4 задания:
1. Вам необходимо реализовать SVD разложения используя SGD на explicit данных
2. Вам необходимо реализовать матричное разложения используя ALS на implicit данных
3. Вам необходимо реализовать матричное разложения используя BPR на implicit данных
4. Вам необходимо реализовать матричное разложения используя WARP на implicit данных

Мягкий дедлайн 13 Октября (пишутся замечания, выставляется оценка, есть возможность исправить до жесткого дедлайна)

Жесткий дедлайн 20 Октября (Итоговая проверка)

In [1]:
cd ~/GitHub/RecSys-hse-fall-2021

/Users/diat.lov/GitHub/RecSys-hse-fall-2021


In [2]:
!pwd

/Users/diat.lov/GitHub/RecSys-hse-fall-2021


In [3]:
%load_ext autoreload
%autoreload 2

import implicit
import pandas as pd
import numpy as np
import scipy.sparse as sp
import sys
import scipy

from src.hw1.resources import DATA_PATH
from src.hw1.mse_logs import LOGGING_PATH

from src.hw1.util import get_csr_matrix_from_pdf

В данной работе мы будем работать с explicit датасетом movieLens, в котором представленны пары user_id movie_id и rating выставленный пользователем фильму

Скачать датасет можно по ссылке https://grouplens.org/datasets/movielens/1m/

In [4]:
ratings = pd.read_csv(f'{DATA_PATH.parent}/ratings.dat', delimiter='::', header=None, 
                      names=['user_id', 'movie_id', 'rating', 'timestamp'], 
                      usecols=['user_id', 'movie_id', 'rating'], engine='python')

In [5]:
movie_info = pd.read_csv(f'{DATA_PATH.parent}/movies.dat', delimiter='::', header=None, 
                         names=['movie_id', 'name', 'category'], engine='python')

Explicit данные

In [91]:
ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5
5,1,1197,3
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4


In [293]:
users = ratings["user_id"]
movies = ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

Для того, чтобы преобразовать текущий датасет в Implicit, давайте считать что позитивная оценка это оценка >=4

In [59]:
implicit_ratings = ratings.loc[(ratings['rating'] >= 4)]

In [60]:
implicit_ratings.head(10)

,user_id,movie_id,rating
0,1,1193,5
3,1,3408,4
4,1,2355,5
6,1,1287,5
7,1,2804,5
8,1,594,4
9,1,919,4
10,1,595,5
11,1,938,4
12,1,2398,4


Удобнее работать с sparse матричками, давайте преобразуем DataFrame в CSR матрицы

In [68]:
users = implicit_ratings["user_id"]
movies = implicit_ratings["movie_id"]
user_item = sp.coo_matrix((np.ones_like(users), (users, movies)))
user_item_t_csr = user_item.T.tocsr()
user_item_csr = user_item.tocsr()

В качестве примера воспользуемся ALS разложением из библиотеки implicit

Зададим размерность латентного пространства равным 64, это же определяет размер user/item эмбедингов

In [ ]:
model = implicit.als.AlternatingLeastSquares(factors=64, iterations=100, calculate_training_loss=True)

In [ ]:
model.fit(user_item_t_csr)

В качестве loss здесь всеми любимый RMSE

Построим похожие фильмы по 1 movie_id = Истории игрушек

In [16]:
movie_info.head(5)

,movie_id,name,category
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [17]:
get_similars = lambda item_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                        for x in model.similar_items(item_id)]

Как мы видим, симилары действительно оказались симиларами.

Качество симиларов часто является хорошим способом проверить качество алгоритмов.

P.S. Если хочется поглубже разобраться в том как разные алгоритмы формируют разные латентные пространства, рекомендую загружать полученные вектора в tensorBoard и смотреть на сформированное пространство

In [11]:
get_similars(1, model)

['0    Toy Story (1995)',
 '3045    Toy Story 2 (1999)',
 "2286    Bug's Life, A (1998)",
 '33    Babe (1995)',
 '584    Aladdin (1992)',
 '2315    Babe: Pig in the City (1998)',
 '1838    Mulan (1998)',
 '1526    Hercules (1997)',
 '2618    Tarzan (1999)',
 '2692    Iron Giant, The (1999)']

Давайте теперь построим рекомендации для юзеров

Как мы видим юзеру нравится фантастика, значит и в рекомендациях ожидаем увидеть фантастику

In [12]:
get_user_history = lambda user_id, implicit_ratings : [movie_info[movie_info["movie_id"] == x]["name"].to_string() 
                                            for x in implicit_ratings[implicit_ratings["user_id"] == user_id]["movie_id"]]

In [13]:
get_user_history(4, implicit_ratings)

['3399    Hustler, The (1961)',
 '2882    Fistful of Dollars, A (1964)',
 '1196    Alien (1979)',
 '1023    Die Hard (1988)',
 '257    Star Wars: Episode IV - A New Hope (1977)',
 '1959    Saving Private Ryan (1998)',
 '476    Jurassic Park (1993)',
 '1180    Raiders of the Lost Ark (1981)',
 '1885    Rocky (1976)',
 '1081    E.T. the Extra-Terrestrial (1982)',
 '3349    Thelma & Louise (1991)',
 '3633    Mad Max (1979)',
 '2297    King Kong (1933)',
 '1366    Jaws (1975)',
 '1183    Good, The Bad and The Ugly, The (1966)',
 '2623    Run Lola Run (Lola rennt) (1998)',
 '2878    Goldfinger (1964)',
 '1220    Terminator, The (1984)']

Получилось! 

Мы действительно порекомендовали пользователю фантастику и боевики, более того встречаются продолжения тех фильмов, которые он высоко оценил

In [14]:
get_recommendations = lambda user_id, model : [movie_info[movie_info["movie_id"] == x[0]]["name"].to_string() 
                                               for x in model.recommend(user_id, user_item_csr)]

In [15]:
get_recommendations(4, model)

['585    Terminator 2: Judgment Day (1991)',
 '1271    Indiana Jones and the Last Crusade (1989)',
 '1182    Aliens (1986)',
 '1284    Butch Cassidy and the Sundance Kid (1969)',
 '2502    Matrix, The (1999)',
 '1178    Star Wars: Episode V - The Empire Strikes Back...',
 '1892    Rain Man (1988)',
 '3402    Close Encounters of the Third Kind (1977)',
 '1179    Princess Bride, The (1987)',
 '847    Godfather, The (1972)']

Теперь ваша очередь реализовать самые популярные алгоритмы матричных разложений

Что будет оцениваться:
1. Корректность алгоритма
2. Качество получившихся симиларов
3. Качество итоговых рекомендаций для юзера

### Задание 1. Не использую готовые решения, реализовать SVD разложение используя SGD на explicit данных

In [263]:
from src.hw1.models.svd import SVD

In [7]:
user_item_csr = get_csr_matrix_from_pdf(ratings)

In [39]:
svd_model = SVD(user_item_csr)
user_matrix, item_matrix = svd_model.fit(logging_path=LOGGING_PATH.parent)

Start fitting the model...
Wow, 0th is fitted in 0 minutes!
Epoch: 0, mse: 0.9116954179450224.
Wow, 1th is fitted in 0 minutes!
Epoch: 1, mse: 0.8561389733574772.
Wow, 2th is fitted in 1 minutes!
Epoch: 2, mse: 0.8245812195509796.
Wow, 3th is fitted in 1 minutes!
Epoch: 3, mse: 0.7855372771496444.
Wow, 4th is fitted in 2 minutes!
Epoch: 4, mse: 0.7494035759752105.
Wow, 5th is fitted in 2 minutes!
Epoch: 5, mse: 0.7195320250322643.
Wow, 6th is fitted in 3 minutes!
Epoch: 6, mse: 0.6958893526811075.
Wow, 7th is fitted in 3 minutes!
Epoch: 7, mse: 0.6754479383837523.
Wow, 8th is fitted in 3 minutes!
Epoch: 8, mse: 0.6604057282532552.
Wow, 9th is fitted in 4 minutes!
Epoch: 9, mse: 0.6479552741505301.
Wow, 10th is fitted in 4 minutes!
Epoch: 10, mse: 0.6377475514404695.
Wow, 11th is fitted in 5 minutes!
Epoch: 11, mse: 0.6280859584750977.
Wow, 12th is fitted in 5 minutes!
Epoch: 12, mse: 0.6226049533640118.
Wow, 13th is fitted in 6 minutes!
Epoch: 13, mse: 0.6159023857654081.
Wow, 14th is 

##### Посмотрим на топ-5 похожих фильмов на Историю игрушек и Индиану-Джонс:

In [61]:
simillar_to_toy_story, similar_to_indiana = svd_model.get_k_similar_movies(np.array([1, 1291]))

In [62]:
movie_info.set_index('movie_id').loc[simillar_to_toy_story]

,name,category
movie_id,,
3114,Toy Story 2 (1999),Animation|Children's|Comedy
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
2090,"Rescuers, The (1977)",Animation|Children's
1566,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
588,Aladdin (1992),Animation|Children's|Comedy|Musical


In [63]:
movie_info.set_index('movie_id').loc[similar_to_indiana]

,name,category
movie_id,,
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War
260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Fantasy|Sci-Fi
1198,Raiders of the Lost Ark (1981),Action|Adventure
1196,Star Wars: Episode V - The Empire Strikes Back...,Action|Adventure|Drama|Sci-Fi|War
1396,Sneakers (1992),Crime|Drama|Sci-Fi


### Задание 2. Не использую готовые решения, реализовать матричное разложение используя ALS на implicit данных

In [67]:
from src.hw1.models.als import ALS

In [68]:
user_item_csr = get_csr_matrix_from_pdf(ratings, implicit=True)

als = ALS(user_item_csr, 16)

In [253]:
user_matrix, item_matrix = als.fit_matrix_completion(gamma=100, epochs=2)

Mse on epoch 0: 0.9999431825010032.
Mse on epoch 1: 1.0.
Model fitted in: 2 seconds.


Комментарий, почему такой mse странный: 

В первой реализации использовала формулы из ALS for Matrix Completion (стр 2: http://stanford.edu/~rezab/classes/cme323/S15/notes/lec14.pdf), 
модель получилась супер чувствиетлена к выбору параметров – hiden_dim и gamma. 

Dot матриц пользователей и айтемов может варьироваться в диапазоне [-1e-32, +1e16] :(

С выбранными мной параметрами – это [4e-1 ~1e-16], пробовала как-то менять гамму на новых эпохах, чтобы не было overflow и предсказания были ближе к диапазону [0, 1], но все это дает хуже similar'ы чем текущий вариант.

In [257]:
simillar_to_toy_story, similar_to_indiana = als.get_k_similar_movies(np.array([1, 1291]))

In [258]:
movie_info.set_index('movie_id').loc[simillar_to_toy_story]

,name,category
movie_id,,
3114,Toy Story 2 (1999),Animation|Children's|Comedy
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
364,"Lion King, The (1994)",Animation|Children's|Musical
2762,"Sixth Sense, The (1999)",Thriller


In [259]:
movie_info.set_index('movie_id').loc[similar_to_indiana]

,name,category
movie_id,,
592,Batman (1989),Action|Adventure|Crime|Drama
2000,Lethal Weapon (1987),Action|Comedy|Crime|Drama
1036,Die Hard (1988),Action|Thriller
1610,"Hunt for Red October, The (1990)",Action|Thriller
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War


Просто матричный ALS без итераций по пользователям и фильмам:

In [69]:
user_matrix, item_matrix = als.fit()

Mse on epoch 0: 0.6446789274584319.
Mse on epoch 5: 0.509219461903636.
Mse on epoch 10: 0.508385938828407.
Mse on epoch 15: 0.5081778601596341.
Mse on epoch 20: 0.5080042471788881.
Mse on epoch 25: 0.5078764975424042.
Model fitted in: 15 seconds.


In [50]:
simillar_to_toy_story, similar_to_indiana = als.get_k_similar_movies(np.array([1, 1291]))

In [51]:
movie_info.set_index('movie_id').loc[simillar_to_toy_story]

,name,category
movie_id,,
3114,Toy Story 2 (1999),Animation|Children's|Comedy
588,Aladdin (1992),Animation|Children's|Comedy|Musical
34,Babe (1995),Children's|Comedy|Drama
2355,"Bug's Life, A (1998)",Animation|Children's|Comedy
595,Beauty and the Beast (1991),Animation|Children's|Musical


In [52]:
movie_info.set_index('movie_id').loc[similar_to_indiana]

,name,category
movie_id,,
1198,Raiders of the Lost Ark (1981),Action|Adventure
2115,Indiana Jones and the Temple of Doom (1984),Action|Adventure
1036,Die Hard (1988),Action|Thriller
592,Batman (1989),Action|Adventure|Crime|Drama
1210,Star Wars: Episode VI - Return of the Jedi (1983),Action|Adventure|Romance|Sci-Fi|War


В этой реализации similar'ы похожие, но реализация более стабльна и числа в получившихся матрицах больше похожи на правду.

Посчитаем recall@100, recall@25, recall@5:

In [99]:
user_ids = np.random.choice(ratings.user_id.unique(), 1000)
predictions_100 = als.recommend(user_ids, n=100)
predictions_25 = als.recommend(user_ids, n=25)
predictions_5 = als.recommend(user_ids, n=5)

np.round(als.recall(user_ids, predictions_100), 2), np.round(als.recall(user_ids, predictions_25), 2), np.round(als.recall(user_ids, predictions_5), 2)

(0.52, 0.25, 0.08)

### Задание 3. Не использую готовые решения, реализовать матричное разложение BPR на implicit данных

In [100]:
from src.hw1.models.bpr import BPR

In [101]:
user_item_csr = get_csr_matrix_from_pdf(ratings, implicit=True)
bpr = BPR(user_item_csr, 16)

In [102]:
user_matrix, item_matrix = bpr.fit()

Error on epoch 0: 0.5036435905929115.
Error on epoch 10: 0.4368996356409407.
Error on epoch 20: 0.38257701225571383.
Error on epoch 30: 0.3347134812851938.
Error on epoch 40: 0.30225240145743626.
Model is fitted in 44 seconds.


In [103]:
simillar_to_toy_story, similar_to_indiana = bpr.get_k_similar_movies(np.array([1, 1291]))

In [104]:
movie_info.set_index('movie_id').loc[simillar_to_toy_story]

,name,category
movie_id,,
912,Casablanca (1942),Drama|Romance|War
3296,To Sir with Love (1967),Drama
3750,Boricua's Bond (2000),Drama
1510,"Brother's Kiss, A (1997)",Drama
1843,Slappy and the Stinkers (1998),Children's|Comedy


In [105]:
movie_info.set_index('movie_id').loc[similar_to_indiana]

,name,category
movie_id,,
2841,Stir of Echoes (1999),Thriller
977,Moonlight Murder (1936),Mystery
1312,Female Perversions (1996),Drama
3905,"Specials, The (2000)",Comedy
50,"Usual Suspects, The (1995)",Crime|Thriller


Рекомендации получились не очень – не нашла ошибки в алгоритме, посмотрим на топ 20 фильмов с наибольшим количеством оценок:

In [146]:
ratings.groupby("movie_id").count().join(movie_info, on="movie_id", how="inner").sort_values("rating", ascending=False).drop(columns = ["user_id", "movie_id"]).head(20)

,rating,name,category
movie_id,,,
2858,3428,Brief Encounter (1946),Drama|Romance
260,2991,Ladybird Ladybird (1994),Drama
1196,2990,Alien (1979),Action|Horror|Sci-Fi|Thriller
1210,2883,Raging Bull (1980),Drama
480,2672,Lassie (1994),Adventure|Children's
2028,2653,Something Wicked This Way Comes (1983),Children's|Horror
589,2649,"Silence of the Lambs, The (1991)",Drama|Thriller
2571,2590,Superman (1978),Action|Adventure|Sci-Fi
1270,2583,Some Kind of Wonderful (1987),Drama|Romance


На рекомендации most-popular тоже не похоже, посчитаем recall@100, recall@25, recall@5:

In [106]:
user_ids = np.random.choice(ratings.user_id.unique(), 1000)
predictions_100 = bpr.recommend(user_ids, n=100)
predictions_25 = bpr.recommend(user_ids, n=25)
predictions_5 = bpr.recommend(user_ids, n=5)

np.round(bpr.recall(user_ids, predictions_100), 2), np.round(bpr.recall(user_ids, predictions_25), 2), np.round(bpr.recall(user_ids, predictions_5), 2)

(0.24, 0.11, 0.03)

Recall в два раза ниже чем на ALS, но все же что-то адекватное, а вот для поиска ближайших item'ов моя реализвация так себе подходит..

### Задание 4. Не использую готовые решения, реализовать матричное разложение WARP на implicit данных